In [13]:
%autosave 0

Autosave disabled


### Text classification with deep learning

Text classification is one of the most basic tasks we can accomplish within nlp, which makes it a great demonstration for the power of some of the important techniques. Here we will discuss processing text a few different ways to feed our text data into a simple neural network model.

To start, we can accomplish building a model purely with our old transformations - in this case, we are going to create binary vectors of the words that represent the IMDB dataset that the first assignment was based on:

In [2]:
from __future__ import print_function

import numpy as np
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras.datasets import imdb

max_features = 1000
batch_size = 32
epochs = 5

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


Because this is a binary classification (positive or negative reviews), you should get 2 classes back:

In [3]:
num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

2 classes


In [4]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
x_train shape: (25000, 1000)
x_test shape: (25000, 1000)


In [9]:
print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (25000, 2)
y_test shape: (25000, 2)


In [10]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Building model...
Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 235s 10ms/step - loss: 0.3902 - acc: 0.8292 - val_loss: 0.3439 - val_acc: 0.8524
Epoch 2/5
22500/22500 [==============================] - 8s 338us/step - loss: 0.3087 - acc: 0.8673 - val_loss: 0.3747 - val_acc: 0.8392
Epoch 3/5
22500/22500 [==============================] - 6s 269us/step - loss: 0.2677 - acc: 0.8887 - val_loss: 0.3375 - val_acc: 0.8628
Epoch 4/5
22500/22500 [==============================] - 6s 264us/step - loss: 0.2113 - acc: 0.9149 - val_loss: 0.3515 - val_acc: 0.8568
Epoch 5/5
25000/25000 [==============================] - 3s 104us/step
Test score: 0.3607002733755112
Test accuracy: 0.85512


## Using embeddings
https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

### Embeddings built into Keras

In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [14]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 166s 7ms/step - loss: 0.4600 - acc: 0.7818 - val_loss: 0.3923 - val_acc: 0.8266
Epoch 2/5
25000/25000 [==============================] - 170s 7ms/step - loss: 0.2974 - acc: 0.8772 - val_loss: 0.3728 - val_acc: 0.8366
Epoch 3/5
25000/25000 [==============================] - 169s 7ms/step - loss: 0.2200 - acc: 0.9133 - val_loss: 0.4065 - val_acc: 0.8330
Epoch 4/5
25000/25000 [==============================] - 169s 7ms/step - loss: 0.1527 - acc: 0.9418 - val_loss: 0.4544 - val_acc: 0.8257
Epoch 5/5
25000/25000 [==============================] - 32s 1ms/step
Test score: 0.5739546837425232
Test accuracy: 0.82356


### Exercise: Use downloaded embeddings (either Glove or word2vec)  to build a model to predict the sentiment of a movie review

In [22]:
from __future__ import print_function

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.datasets import imdb

vocab_size = len(imdb.get_word_index().items())
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)



Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [27]:
# load the whole embedding into memory
embeddings_index = dict()
try:
    f = open('data/glove.6B/glove.6B.300d.txt', encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
finally:
    f.close()
print('Loaded %s word vectors.' % len(embeddings_index))



Loaded 400000 word vectors.


In [38]:
len(embeddings_index['the'])


300

In [40]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in imdb.get_word_index().items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
        
# define model
print('Build model...')
model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], 
              input_length=maxlen, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(x_test, y_test))

score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 80, 300)           26575200  
_________________________________________________________________
flatten_1 (Flatten)          (None, 24000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 24001     
Total params: 26,599,201
Trainable params: 24,001
Non-trainable params: 26,575,200
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 10s 394us/step - loss: 0.7394 - acc: 0.5669 - val_loss: 0.7486 - val_acc: 0.5847
Epoch 2/5
25000/25000 [==============================] - 9s 360us/step - loss: 0.5788 - acc: 0.7043 - val_loss: 0.7629 - val_acc: 0.5982
Epoch 3/5
25000/25000 [=====================